导入必要的库函数，这里主要用的是sklearn、pandas、matplotlib、numpy

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import numpy as np

清洗数据 我清洗的我认为无关的数据，数据集在github文件夹里，需要下载

In [ ]:
data = pd.read_csv(r"你存这个数据集的路径\high_diamond_ranked_10min.csv")
#这个很重要的哦
data.drop(["gameId"
           ,"blueWardsPlaced"
           ,"blueWardsDestroyed"
           #,"blueAssists"
           #,"blueEliteMonsters"
           ,"blueAvgLevel"
           ,"blueTotalJungleMinionsKilled"
           ,"blueCSPerMin"
           ,"blueGoldPerMin"
           ,"blueGoldDiff"
           ,"blueExperienceDiff"
           ,"blueTotalExperience"
           ,"redWardsPlaced"
           ,"redWardsDestroyed"
           ,"redFirstBlood"
           ,"redKills"
           ,"redDeaths"
           #,"redAssists"
           #,"redEliteMonsters"
           ,"redAvgLevel"
           ,"redTotalJungleMinionsKilled"
           ,"redGoldDiff"
           ,"redExperienceDiff"
           ,"redTotalExperience"
           ,"redCSPerMin"
           ,"redGoldPerMin"],inplace = True ,axis = 1)

x = data.iloc[:,data.columns != "blueWins"] #数据的特征
y = data.iloc[:,data.columns == "blueWins"] #数据的标签

Xtrain, Xtest, Ytrain, Ytest = train_test_split(x,y,test_size = 0.3)

for i in [Xtrain, Xtest, Ytrain, Ytest]:
    i.index = range(i.shape[0])
lol = DecisionTreeClassifier(random_state=0)#实例化
lol = lol.fit(Xtrain,Ytrain)

score_ = lol.score(Xtest,Ytest)
score_#考虑是不是要剪枝，纯度太低了，剪枝吧

score = cross_val_score(lol,x,y,cv = 10).mean()
score#精确度很低

决策树的关键在于剪枝,先看看有没有过拟合

In [ ]:
tr = []
te = []
for i in range(30):
    lol = DecisionTreeClassifier(random_state=0
                                ,max_depth=i+1
                                #,criterion="entropy" #用不用基尼系数的影响不大
                                )
    lol = lol.fit(Xtrain,Ytrain)
    score_tr = lol.score(Xtrain,Ytrain)
    score_te = cross_val_score(lol,x,y,cv = 10).mean()
    tr.append(score_tr)
    te.append(score_te)
print(max(te))
plt.plot(range(1,31),tr,color = "red",label = "train")
plt.plot(range(1,31),te,color = "blue",label = "test")
plt.xticks(range(1,31))
plt.legend()
plt.show()

利用网格搜索技术——暴力破解得到相对最优参数

In [ ]:
import numpy as np
gini_thresholds = np.linspace(0,0.5,20)

parameters = {'splitter':('best','random')
             ,'criterion':('gini','entropy')
             ,'max_depth':[5]#刚才最大深度已经搜索出来就是5
             ,'min_samples_leaf':[*range(1,50,5)]
             ,'min_impurity_decrease':[*np.linspace(0,0.5,20)]
             }
lol = DecisionTreeClassifier(random_state=0)
GS = GridSearchCV(lol,parameters,cv = 10)
GS.fit(Xtrain, Ytrain)

GS.best_params_

GS.best_score_
#可以得到最终的决策树参数

用网格参数得到最佳参数，得到最后的训练模型

In [ ]:
lol = DecisionTreeClassifier(random_state=0
                            ,criterion = 'entropy'
                            ,max_depth = 5
                            ,min_impurity_decrease = 0.0
                            ,min_samples_leaf = 46
                            ,splitter ='best')#实例化
lol = lol.fit(Xtrain,Ytrain)

输入输出阶段

In [ ]:
import csv
lis = ['blueFirstBlood'
                 ,'blueKills'
                 ,'blueDeaths'
                 ,'blueAssists'
                 ,'blueEliteMonsters'
                 ,'blueDragons'
                 ,'blueHeralds'
                 ,'blueTowersDestroyed'
                 ,'blueTotalGold'
                 ,'blueTotalMinionsKilled'
                 ,'redAssists'
                 ,'redEliteMonsters'
                 ,'redDragons'
                 ,'redHeralds'
                 ,'redTowersDestroyed'
                 ,'redTotalGold'
                 ,'redTotalMinionsKilled']
header = np.array(lis)

lol_data = []
for i in range(17):
    print("Please Iuput "+lis[i]+" : ",end='')
    s = int(input())
    lol_data.append([s])
    
lol_data = np.array(lol_data)

with open('lol.csv','w',newline='') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(header)
    f_csv.writerows(lol_data.T)

want_data = pd.read_csv(r'lol.csv')

print()
if lol.predict(want_data) == 1:
    print("BlueTeamWin, Probability: "+str(lol.predict_proba(want_data)[0][1]))
else:
    print("BlueTeamLose, Probability: "+str(lol.predict_proba(want_data)[0][0]))

import os
os.remove('lol.csv')